In [1]:
# 导入必要的库并初始化模型
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from datasets import load_dataset
import evaluate
import numpy as np
import os
import wandb

# 解决 tokenizers 警告
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 登录 wandb - 第一次运行时需要输入 API key
# 获取 API key: https://wandb.ai/authorize
wandb.login()

# 加载 MRPC 数据集
dataset = load_dataset("glue", "mrpc")

# 加载预训练模型和分词器
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 数据预处理函数
def preprocess_function(examples):
    return tokenizer(
        examples["sentence1"], 
        examples["sentence2"], 
        truncation=True, 
        padding="max_length",
        max_length=128
    )

# 对数据集进行分词
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 创建 data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 定义评估指标
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"]
    }

print("✅ 模型和数据集已准备就绪！")
print("📊 wandb 已登录，运行下一个 cell 开始训练并查看链接")

/Users/haoyiwen/miniforge3/envs/d2l/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
wandb: Currently logged in as: 1552975589 (1552975589-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

✅ 模型和数据集已准备就绪！
📊 wandb 已登录，运行下一个 cell 开始训练并查看链接


In [2]:
# 开始训练并使用 wandb 追踪实验
from transformers import Trainer, TrainingArguments
import wandb

# 初始化 wandb 项目（会生成实验链接）
run = wandb.init(
    project="transformer-fine-tuning",  # 项目名称
    name="bert-mrpc-training",          # 本次运行的名称
    config={                             # 记录超参数
        "model": "bert-base-uncased",
        "dataset": "MRPC",
        "epochs": 3,
        "batch_size": 16,
        "learning_rate": 2e-5
    }
)

print("🔗 wandb 运行链接将在训练开始后显示...")
print("-" * 50)

# 配置训练参数
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",           # 每 N 步评估一次
    eval_steps=50,                   # 每 50 步评估
    save_steps=100,                  # 每 100 步保存模型
    logging_steps=10,                # 每 10 步记录一次
    num_train_epochs=3,              # 训练 3 个 epoch
    per_device_train_batch_size=16,  # 批次大小
    per_device_eval_batch_size=16,
    learning_rate=2e-5,              # 学习率
    warmup_steps=100,                # 预热步数
    load_best_model_at_end=True,    # 训练结束后加载最佳模型
    metric_for_best_model="f1",     # 用 F1 分数选择最佳模型
    report_to="wandb",               # 报告给 wandb
    logging_first_step=True,        # 记录第一步（看初始状态）
)

# 创建 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 开始训练！
print("🚀 开始训练...")
print("📊 打开上方显示的 wandb 链接查看实时图表")
print("-" * 50)

trainer.train()

# 训练完成
print("\n✅ 训练完成！")
print(f"📈 查看完整结果: {run.get_url()}")

# 评估最终模型
final_metrics = trainer.evaluate()
print("\n📊 最终评估指标:")
for key, value in final_metrics.items():
    print(f"  {key}: {value:.4f}")

# 结束 wandb 运行
wandb.finish()

🔗 wandb 运行链接将在训练开始后显示...
--------------------------------------------------


/var/folders/gz/0gxv2qld2094b0b1mp3z79tc0000gn/T/ipykernel_91379/4175999667.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 开始训练...
📊 打开上方显示的 wandb 链接查看实时图表
--------------------------------------------------


/Users/haoyiwen/miniforge3/envs/d2l/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.675700,0.641212,0.708333,0.822653
100,0.558800,0.573521,0.715686,0.826347
150,0.531600,0.514748,0.772059,0.851200
200,0.519000,0.498986,0.769608,0.851735
250,0.328400,0.384477,0.835784,0.885470
300,0.317800,0.418001,0.816176,0.875622
350,0.327600,0.386038,0.833333,0.877698
400,0.237600,0.404615,0.848039,0.897351
450,0.319200,0.343997,0.857843,0.897887
500,0.180800,0.369759,0.857843,0.897527


/Users/haoyiwen/miniforge3/envs/d2l/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/haoyiwen/miniforge3/envs/d2l/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/haoyiwen/miniforge3/envs/d2l/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/haoyiwen/miniforge3/envs/d2l/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/h


✅ 训练完成！
📈 查看完整结果: https://wandb.ai/1552975589-none/transformer-fine-tuning/runs/6zk7aejv


/Users/haoyiwen/miniforge3/envs/d2l/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



📊 最终评估指标:
  eval_loss: 0.4029
  eval_accuracy: 0.8603
  eval_f1: 0.9016
  eval_runtime: 3.0678
  eval_samples_per_second: 132.9930
  eval_steps_per_second: 8.4750
  epoch: 3.0000


eval/accuracy,▁▁▄▄▇▆▇▇██▇███
eval/f1,▁▁▃▃▆▆▆▇▇▇▇███
eval/loss,█▆▅▅▂▃▂▂▁▂▂▂▂▂
eval/runtime,▆▇▇▇▇▇▇▇█▇▂▂▁▂
eval/samples_per_second,▃▂▂▂▂▂▂▂▁▂▇▇█▇
eval/steps_per_second,▃▂▂▂▂▂▂▂▁▂▇▇█▇
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█████
train/grad_norm,▂▂▂▃▃▂▂▂▂▂▂▂▅▂▃▂▃▃▄▂▄▆▂▃▆▃▂▆▃▃█▂▄▅▆▅▃▃▂▁
train/learning_rate,▁▂▄▆▇█████▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▂▂▂▂▂▂▂▁▁
+1,...
